In [1]:
import os
import numpy as np
import pandas as pd
import seaborn as sns

from pydub import AudioSegment, effects
from pydub.generators import WhiteNoise
from pydub.playback import play
from pydub.utils import mediainfo
import librosa
from librosa import display   
import noisereduce as nr
import IPython.display as ipd
from IPython.display import Audio
from IPython.display import clear_output
import matplotlib.pyplot as plt
import pytz
import cv2

import tensorflow as tf
import tensorboard
from sklearn import metrics
from sklearn.preprocessing import LabelEncoder
from json_tricks import dump, load

from DataModel import DataModel
from SERModel import SERModel
from Evaluation import Evaluation

from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

print("TensorFlow version:  ", tf.__version__)
print("TensorBoard version: ", tensorboard.__version__)

tz = pytz.timezone('Asia/Hong_Kong')

TensorFlow version:   2.7.0
TensorBoard version:  2.10.0


In [2]:
testing = SERModel("optimal", "fjdslk")
testing.summary()


########################################################
################### Training Section ###################
########################################################

Model Information:
    Model Choice     : optimal
    Experiment Name  : 12-28 00h40m27s fjdslk
    Log Directory    : /Users/alexto/Documents/Programming/HKU/FYP/Speech Emotion Recognition/IEMOCAP_ModelLog/12-28 00h40m27s fjdslk
    Result Directory : /Users/alexto/Documents/Programming/HKU/FYP/Speech Emotion Recognition/IEMOCAP_TrainedModel/12-28 00h40m27s fjdslk
    Optimizer        : Adam
      Learning Rate  : 0.0001
      Decay          : 0.001
    Loss             : Sparse Categorical Crossentropy
    Metrics          : Accuracy

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 128, 251, 32)      320       
                                                                 


2022-12-28 00:40:28.011898: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
labelsToInclude = ['Neutral', 'Frustration', 'Anger', 'Sadness', 'Happiness']
mergeHappinessExcitement = True
splitDuration = 8
ignoreDuration = 2
transformByStft=True
hop_length = 512
win_length = 2048
n_mels = 128

mixDataModel5LabelsSplit4Ignore2Stft = DataModel(labelsToInclude=labelsToInclude,
                                                mergeHappinessExcitement=mergeHappinessExcitement,
                                                splitDuration=splitDuration,
                                                ignoreDuration=ignoreDuration,
                                                transformByStft=transformByStft,
                                                hop_length=hop_length,
                                                win_length=win_length,
                                                n_mels=n_mels)
mixDataModel5LabelsSplit4Ignore2Stft.extractIEMOCAPData()
mixDataModel5LabelsSplit4Ignore2Stft.extractEmoDBData()
mixDataModel5LabelsSplit4Ignore2Stft.processData()

Loading and Extracting EmoDB Data...
    Loaded and Extracted   408 data

Data Extration Completed
    Number of data: 408
      Neutral     : 79
      Frustration : 69
      Anger       : 127
      Sadness     : 62
      Happiness   : 71
      Excitement  : 0
      Surprise    : 0
      Disgust     : 0
      Fear        : 0
      Boredom     : 0

Splitting data...
Train Test Split Completed
    Training Size : 327
    Testing Size  : 81

Split or Add Padding for training data:
    Split Duration  : 8
    Ignore Duration : 2
Processing...
    Processed   327 data split and padding
Data Splitting and Padding For Training Completed!

Processing training data to Mel Spectrogram...
    Processed   327 Mel Spectrogram
Mel Spectrogram Processing For Training Completed
    Shape of training images: (327, 128, 251, 1)

Processing training labels...
Label Processing For Training Completed

Split or Add Padding for testing data
    Split Duration  : 8
    Ignore Duration : 2
Processing...
    Pr

In [ ]:
def trainingOptimal(dataModel, experimentName, modelName, epochs):
  ySize = len(dataModel.labels_name)

  cnnModel = SERModel(modelName, experimentName, ySize=ySize)
  history = cnnModel.fit(dataModel.x_train, dataModel.y_train, epochs, dataModel.validation_percent)
  
  evaluation = Evaluation(dataModel, cnnModel.resultDir, cnnModel.logDir, model=cnnModel.model)
  evaluation.evaluateAllHistory(history)
  
  print('')
  print('File Name: ' + cnnModel.logDir.split('/')[-1])

experimentName = "(Experiment 16) CNN LSTM Model C (200 Epochs) (IEMOCAP EmoDB) (No Data Aug) (5 Emotions with Merge and Split 4 Ignore 2) (0001 lr 001 decay 32 batchsize)"
experimentName = "Optimal"
modelName = "optimal"
epochs = 200

trainingOptimal(mixDataModel5LabelsSplit4Ignore2Stft, experimentName, modelName, epochs)